In [123]:
from align import *

In [124]:
manifest_loc = '/grid/home/nbourgeois/wip/ampliseq_compare/amplicon_manifest.txt'
alignment_path = '/grid/home/nbourgeois/nbourgeois_scripts/snakemake/miseq/tests/ampliseq_run_2201/data/aligned/HUM180201A_S4.bam'

ampliseq_manifest = pd.read_csv(manifest_loc,sep='\t',skiprows=9)
ampliseq_manifest = ampliseq_manifest.set_index('Target ID')

In [125]:
def clean_transcipt_df(ampliseq_manifest):
    # get transcripts that appear multiple times in the manifest
    df_tran = ampliseq_manifest.reset_index().groupby('Transcript ID').count()
    multi_trans = df_tran.loc[df_tran['Target ID']>1].index.values

    for transcript in multi_trans:
        amp_t = ampliseq_manifest.loc[ampliseq_manifest['Transcript ID']==transcript]
        gene_list = amp_t['Gene']
        if any(~gene_list.isna()):
            gene = gene_list[~gene_list.isna()][0]
            ampliseq_manifest.loc[ampliseq_manifest['Transcript ID']==transcript,'Gene'] = gene
    return(ampliseq_manifest)

In [126]:
df = create_alignment_df(alignment_path)


In [127]:
ser_counts,ser_ambig,ser_metrics = sort_reads(df,verbose=False,)

In [137]:
ser_vals = ser_counts
val_table = ampliseq_manifest[['Gene','Transcript ID']].copy()

# algined to target ref
if len([x for x in ser_vals.index if x in val_table.index]) > 0:
    val_table.loc[ser_vals.index,'val'] = ser_vals.values
    val_table = val_table.loc[~val_table['Gene'].isna()]
    val_table = val_table.fillna(0) 


,Gene,Transcript ID,val
Target ID,,,
T00001,WASH7P,NR_024540,4.0
T00002,OR4F5,NM_001005484,NaN
T00003,LOC729737,NR_039983,NaN
T00004,OR4F3,NM_001005224,NaN
T00005,LOC100288069,NR_033908,NaN
...,...,...,...
T28684,PECAM1,NM_000442,NaN
T28685,RNF5P1,NR_003129,NaN
T28686,SNORA19,NR_002917,1.0
